# Raster Processing Workflow
**Cara Piske, Graduate Program of Hydrologic Sciences, 2022; Advisor: Dr. Adrian Harpold**<br>
<p>This code processes rasterized LAS files. <br>
Lidar data were provided by the Airborne Snow Observatory (ASO), the National Center for Airborne Laser Mapping (NCALM), and Watershed Sciences Inc. (WSI). <br>

The goal of this project is to process snow depth to the one-meter spatial scale while maintaining conservative under-canopy estimates. Therefore, little interpolation occurs under-canopy. We follow these protocols in order to obtain a 1-m rasterized product (as opposed to the 3-m rasterized product provided by ASO on the NSIDC data portal). NCALM and WSI flights were obtained through OpenTopography.

In [1]:
# import necessary packages 
from osgeo import gdal, ogr, osr
import csv
import os
import subprocess
import sys
import pdal
import matplotlib.pyplot as plt
import numpy as np
import json
import glob
import time
# competing paths on our PC 
gdal_merge = os.path.join('C:\\','Users','cpiske','.conda','envs','lidar','Lib','site-packages','osgeo_utils','gdal_merge.py')
gdal_calc = os.path.join('C:\\','Users','cpiske','.conda','envs','lidar','Lib','site-packages','osgeo_utils','gdal_calc.py')
gdal_warp = os.path.join('C:\\','Users','cpiske','.conda','envs','lidar','Lib','site-packages','osgeo_utils','gdal_warp.py')
gdal_polygonize = os.path.join('C:\\','Users','cpiske','.conda','envs','lidar','Lib','site-packages','osgeo_utils','gdal_polygonize.py')
gdal_proximity = os.path.join('C:\\','Users','cpiske','.conda','envs','lidar','Lib','site-packages','osgeo_utils','gdal_proximity.py')

In [2]:
#make sure we're in the right working directory
os.chdir('/')
print(os.getcwd())

G:\


## Classify DEM

### Slope and Aspect

In [3]:
DEM = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/DEM/SCB/NCALM_2014_BE.tif'
slope = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/DEM/SCB/NCALM_2014_slope.tif'
aspect = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/DEM/SCB/NCALM_2014_aspect.tif'
gdal_terrain_cmd = ['gdaldem', 'slope', DEM, slope]
subprocess.run(gdal_terrain_cmd)
gdal_terrain_cmd = ['gdaldem', 'aspect', DEM, aspect]
subprocess.run(gdal_terrain_cmd)

CompletedProcess(args=['gdaldem', 'aspect', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/DEM/SCB/NCALM_2014_BE.tif', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/DEM/SCB/NCALM_2014_aspect.tif'], returncode=0)

### Northness

In [4]:
nness = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/DEM/ASO_2016/NCALM_2014_Nness.tif'

In [5]:
# open slope and aspect as arrays
slope_ras = gdal.Open(slope) # open the file 
aspect_ras = gdal.Open(aspect) # open the file 
# save spatial data
gt = slope_ras.GetGeoTransform()
proj = slope_ras.GetProjection()

slope_arr = slope_ras.GetRasterBand(1).ReadAsArray() #read the first raster band (in this case we know we are only working with single bands) and read to a 2D array
aspect_arr = aspect_ras.GetRasterBand(1).ReadAsArray() #read the first raster band (in this case we know we are only working with single bands) and read to a 2D array

slope_ras = aspect_ras = None # close rasters

In [6]:
# Manipulate rasters and run raster calculations
slope_arr[slope_arr == -9999] = np.nan # assign nans 
aspect_arr[aspect_arr == -9999] = np.nan

slope_rad = np.radians(slope_arr) # convert degrees to radians
aspect_rad = np.radians(aspect_arr)

# see Tennant et al.(cos(aspect)*slope), 2017; Kirchner et al., 2014 (cos(aspect)*sin(slope)); Safa et al. (cos(aspect)*sin(slope), 2020; Molotch et al., 2004 (cos(aspect)*sin(slope)); Broxton et al, (calculated differently)
northness_arr = np.cos(aspect_rad)*np.sin(slope_rad) # calculate northness

In [327]:
# # save to raster
# # export
# driver = gdal.GetDriverByName("GTiff")
# driver.Register()
# outds = driver.Create(nness, xsize = northness_arr.shape[1],
#                       ysize = northness_arr.shape[0], bands = 1, 
#                       eType = gdal.GDT_Float32)
# outds.SetGeoTransform(gt)
# outds.SetProjection(proj)
# outband = outds.GetRasterBand(1)
# outband.WriteArray(northness_arr)
# outband.SetNoDataValue(np.nan)
# outband.FlushCache()

# # close your datasets and bands!!!
# outband = None
# outds = None

## Classify Vegetation
We use vegetation classifications based on specific strata. See: https://docs.google.com/document/d/1-HndVgydwBWQGCzLMCvowkY_Idll0IDog-vd3OrTiio/edit?usp=sharing

In [17]:
# Number of Returns in each HAG strata
strata_neg0pt15_0pt15 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_height_strata/vegStrata_neg0pt15_0pt15.tif'
strata_0pt15_1pt5 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_height_strata/vegStrata_0pt15_1pt5.tif'
strata_1pt5_3 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_height_strata/vegStrata_1pt5_3.tif'
strata_3 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_height_strata/vegStrata_3.tif'
# Filter based off criteria for vegetation classes, files are binary
NOR_neg0pt15_0pt15_GT0 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/counts_filtered/NOR_neg0pt15_0pt15_GT0.tif'
NOR_neg0pt15_0pt15_GTET0 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/counts_filtered/NOR_neg0pt15_0pt15_GTET0.tif'
NOR_0pt15_1pt5_GT0 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/counts_filtered/NOR_0pt15_1pt5_GT0.tif'
NOR_0pt15_1pt5_ET0 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/counts_filtered/NOR_0pt15_1pt5_ET0.tif'
NOR_1pt5_3_ET0 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/counts_filtered/NOR_1pt5_3_ET0.tif'
NOR_3_ET0 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/counts_filtered/NOR_3_ET0.tif'
NOR_3_GT0 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/counts_filtered/NOR_GT0.tif'
# Vegetation Classes, files are binary
ncalm_open = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/NCALM_SCB_2020_open.tif'
ncalm_tall = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/NCALM_SCB_2020_tall.tif'
ncalm_short = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/NCALM_SCB_2020_short.tif'
ncalm_understory = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/NCALM_SCB_2020_understory.tif'

ncalm_short_HAG = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/NCALM_SCB_2020_short_HAG.tif'
ncalm_understory_HAG = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/NCALM_SCB_2020_understory_HAG.tif'

CHM_LT1pt5 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/CHM/NCALM_2014_CHM_LT1pt5.tif'
# veg_OpenTall = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/NCALM_SCB_2014_tallORopen.tif'
# veg_total = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/NCALM_SCB_2014_shortORtallORopen.tif'

veg_classified = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/NCALM_SCB_2014_vegClassified.tif'

We'll start off by looking at the number of returns in each height strata and filtering based off of our criteria. <br>

In [ ]:
# nor_filtered_cmd = ['python', gdal_calc, '-A', strata_neg0pt15_0pt15, '--outfile', NOR_neg0pt15_0pt15_GT0, '--calc="A>0"','--overwrite']
# nor_filtered_cmd = ['python', gdal_calc, '-A', strata_neg0pt15_0pt15, '--outfile', NOR_neg0pt15_0pt15_GTET0, '--calc="A>=0"','--overwrite']
# nor_filtered_cmd = ['python', gdal_calc, '-A', strata_0pt15_1pt5, '--outfile', NOR_0pt15_1pt5_GT0, '--calc="A>0"','--overwrite']
# nor_filtered_cmd = ['python', gdal_calc, '-A', strata_0pt15_1pt5, '--outfile', NOR_0pt15_1pt5_ET0, '--calc="A==0"','--overwrite']
# nor_filtered_cmd = ['python', gdal_calc, '-A', strata_1pt5_3, '--outfile', NOR_1pt5_3_ET0, '--calc="A==0"','--overwrite']
# nor_filtered_cmd = ['python', gdal_calc, '-A', strata_3, '--outfile', NOR_3_ET0, '--calc="A==0"','--overwrite']
# nor_filtered_cmd = ['python', gdal_calc, '-A', strata_3, '--outfile', NOR_3_GT0, '--calc="A>0"','--overwrite']

# subprocess.run(nor_filtered_cmd)

### Open, Tall, Short Veg, Understory
After filtering based of strata criteria, we can classify the snow-off flight into vegetation classifications. <br> 


In [188]:
# open_cmd = ['python', gdal_calc, '-A', NOR_neg0pt15_0pt15_GTET0, '-B', NOR_0pt15_1pt5_ET0,'-C',NOR_1pt5_3_ET0,'-D',NOR_3_ET0,'--NoDataValue','-9999',
#  '--outfile', ncalm_open, '--calc="A*B*C*D"','--overwrite']
# subprocess.run(open_cmd)

# tall_cmd = ['python', gdal_calc, '-A', NOR_neg0pt15_0pt15_GT0, '-B', NOR_0pt15_1pt5_ET0,'-C',NOR_1pt5_3_ET0,'-D',NOR_3_GT0,'--NoDataValue','-9999',
#  '--outfile', ncalm_tall, '--calc="A*B*C*D"','--overwrite']
# subprocess.run(tall_cmd)

# short_cmd = ['python', gdal_calc, '-A', NOR_neg0pt15_0pt15_GTET0, '-B', NOR_0pt15_1pt5_GT0,'-C',NOR_1pt5_3_ET0,'-D',NOR_3_ET0,'--NoDataValue','-9999',
#  '--outfile', ncalm_short, '--calc="A*B*C*D"','--overwrite']
# subprocess.run(short_cmd)

# understory_cmd = ['python', gdal_calc, '-A', NOR_neg0pt15_0pt15_GTET0, '-B', NOR_0pt15_1pt5_GT0,'-C',NOR_1pt5_3_ET0,'-D',NOR_3_GT0,'--NoDataValue','-9999',
#  '--outfile', ncalm_understory, '--calc="A*B*C*D"','--overwrite']
# subprocess.run(understory_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/veg_classes/nor_filtered/NOR_neg0pt15_0pt15_GTET0.tif', '-B', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/veg_classes/nor_filtered/NOR_0pt15_1pt5_GT0.tif', '-C', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/veg_classes/nor_filtered/NOR_1pt5_3_ET0.tif', '-D', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/veg_classes/nor_filtered/NOR_3_ET0.tif', '--NoDataValue', '-9999', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/NCALM_SCB_2014_short.tif', '--calc="A*B*C*D"', '--overwrite'], returncode=0)

Now let's extract the HAG values at potential short/understory locations

In [135]:
NCALM_2014_CHM_LT1pt5 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/CHM/NCALM_2014_CHM_LT1pt5.tif'

In [573]:
# short_cmd = ['python', gdal_calc, '-A', ncalm_short, '-B', CHM_LT1pt5,
#  '--outfile', ncalm_short_HAG, '--calc="B*A"','--overwrite',]
# subprocess.run(short_cmd)

# understory_cmd = ['python', gdal_calc, '-A', ncalm_understory, '-B', CHM_LT1pt5,'--NoDataValue','-9999',
#  '--outfile', ncalm_understory_HAG, '--calc="B*(A==1)"','--overwrite']
# subprocess.run(understory_cmd)

In [172]:
# veg_classified_cmd = ['python', gdal_calc, '-A', ncalm_short, '-B', ncalm_tall,'-C',ncalm_short,'-D',ncalm_understory,'--NoDataValue','-9999',
#  '--outfile', veg_classified, '--calc="1*A+2*B+3*C+4*D"','--overwrite']
# subprocess.run(veg_classified_cmd)


In [ ]:
# # calculate CHM from DSM and DTM
# input_snow_depth = 'SCB/random_forest_data/ASO_sd/ASO_snow_depth_20160417_clp.tif'
# input_snow_density = 'SCB/random_forest_data/snowpalm_density/SP_density_20160417_1m.tif'
# output_swe = 'SCB/random_forest_data/calc_swe/SWE_20160417.tif'
# swe_command = ['gdal_calc.py', '-A', input_snow_depth, '-B', input_snow_density, '--calc="A*B"','--outfile', output_swe]
# subprocess.run(swe_command)

In [20]:
# tall or understory

tall_cmd = ['python', gdal_calc, '-A', ncalm_tall, '-B', ncalm_understory,
 '--outfile', ncalm_tallORunderstory, '--calc="B+A"','--overwrite','--NoDataValue','-9999']
subprocess.run(tall_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/NCALM_SCB_2014_tall.tif', '-B', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/NCALM_SCB_2014_understory.tif', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/NCALM_SCB_2014_tallORunderstory.tif', '--calc="B+A"', '--overwrite', '--NoDataValue', '-9999'], returncode=0)

In [576]:
# short_cmd = ['python', gdal_calc, '-A', ncalm_open, '-B', ncalm_short,
#  '--outfile', ncalm_shortORopen, '--calc="B+A"','--overwrite',]
# subprocess.run(short_cmd)

### Characterize Veg

In [10]:
# Filter out all water bodies
input_shp = 'SCB/supporting_files/masks/NHDWaterbody_2018_EPSG26910.shp'

In [31]:
ncalm_tallORunderstory = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/NCALM_SCB_2014_tallORunderstory.tif'
dist_to_nearest_canopy = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/veg_characteristics/NCALM_SCB_2014_DNC.tif'
dist_to_nearest_canopy_30m = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/veg_characteristics/NCALM_SCB_2014_DNC_30m.tif'
dist_to_nearest_canopy_10m = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/veg_characteristics/NCALM_SCB_2014_DNC_10m.tif'

clump_size = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/veg_characteristics/NCALM_SCB_2014_clumpSize.tif'
clump_size_30 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/veg_characteristics/NCALM_SCB_2014_clumpSize_30m.tif'
clump_size_10 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/veg_characteristics/NCALM_SCB_2014_clumpSize_10m.tif'

In [21]:
# extract vegetation cover (average height and density) across the watershed
proximity_cmd = ['python',gdal_proximity, ncalm_tallORunderstory, dist_to_nearest_canopy,'-values','1','-nodata','-9999']
subprocess.run(proximity_cmd)
burn_command = ['gdal_rasterize', '-b', '1', '-burn', '-9999', input_shp, dist_to_nearest_canopy]
subprocess.call(burn_command)

0

In [22]:
# extract vegetation cover (average height and density) across the watershed
proximity_cmd = ['python',gdal_proximity, ncalm_tallORunderstory, clump_size,'-values','0','-nodata','-9999']
subprocess.run(proximity_cmd)
burn_command = ['gdal_rasterize', '-b', '1', '-burn', '-9999', input_shp, clump_size]
subprocess.call(burn_command)

0

In [26]:
ncalm_fVEG = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/veg_characteristics/NCALM_SCB_2014_fVEG_10m.tif'

In [29]:
# # use ts to specify the x and y extents of bounding box (default 1000)
warp_cmd = ["gdalwarp","-overwrite", ncalm_tallORunderstory, ncalm_fVEG, "-tr", str(10),str(10),'-srcnodata','-9999','-r','average','-dstnodata','-9999']
subprocess.call(warp_cmd)

0

In [32]:
# # use ts to specify the x and y extents of bounding box (default 1000)
# warp_cmd = ["gdalwarp","-overwrite", clump_size, clump_size_30, "-tr", str(30),str(30),'-srcnodata','-9999','-r','average','-dstnodata','-9999']
warp_cmd = ["gdalwarp","-overwrite", clump_size, clump_size_10, "-tr", str(10),str(10),'-srcnodata','-9999','-r','average','-dstnodata','-9999']

subprocess.call(warp_cmd)

0

In [33]:
# # use ts to specify the x and y extents of bounding box (default 1000)
# warp_cmd = ["gdalwarp","-overwrite", dist_to_nearest_canopy, dist_to_nearest_canopy_30m, "-tr", str(30),str(30),'-srcnodata','-9999','-r','average','-dstnodata','-9999']
warp_cmd = ["gdalwarp","-overwrite", dist_to_nearest_canopy, dist_to_nearest_canopy_10m, "-tr", str(10),str(10),'-srcnodata','-9999','-r','average','-dstnodata','-9999']

subprocess.call(warp_cmd)

0

## Calculate Snow Depth

Time to bring the snow in... <br>
For each flight, we have two snow depths, filtered at 3 meters and 5 meters to use for open vs. under canopy <br>
We use the general open/short/tall/understory classifications above but refine for each flight

### Refine Vegetation Classes

In [345]:
# snow_depth_LTET3 = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/corrected_tif/ASO_SCB_20160518_vbc_3.tif'
# snow_depth_LTET5 = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/corrected_tif/ASO_SCB_20160518_vbc_5.tif'
# flight_spec_short = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/veg_classes/ASO_SCB_20160518_short.tif'
# flight_spec_open = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/veg_classes/ASO_SCB_20160518_open.tif'
# flight_spec_understory = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/veg_classes/ASO_SCB_20160518_understory.tif'
# flight_spec_tall = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/veg_classes/ASO_SCB_20160518_tall.tif'

In [383]:
# snow_depth_LTET3 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/corrected_tif/NCALM_SCB_20080210_vbc_3.tif'
# snow_depth_LTET5 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/corrected_tif/NCALM_SCB_20080210_vbc_5.tif'
# flight_spec_short = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/veg_classes/NCALM_SCB_20080210_short.tif'
# flight_spec_open = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/veg_classes/NCALM_SCB_20080210_open.tif'
# flight_spec_understory = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/veg_classes/NCALM_SCB_20080210_understory.tif'
# flight_spec_tall = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/veg_classes/NCALM_SCB_20080210_tall.tif'

Refine vegetation classifications based on snow-on flights. We set a threshold of 30 cm so that if the snow-on flight is <30 cm greater than the snow-off flight, we do not keep that short/understory pixel. 

In [384]:
SD_open_cmd = ['python',gdal_calc, '-A', snow_depth_LTET5, '-B', ncalm_short,'-C', CHM_LT1pt5,'--NoDataValue','-9999',
 '--outfile', flight_spec_short, '--calc="((A-C)>0.3)*B"','--overwrite']
subprocess.run(SD_open_cmd)

SD_open_cmd = ['python',gdal_calc, '-A', flight_spec_short, '-B', ncalm_open,'--NoDataValue','0',
    '--outfile', flight_spec_open, '--calc="1*logical_or(A>0, B>0)"','--overwrite']
subprocess.run(SD_open_cmd)

SD_tall_cmd = ['python',gdal_calc, '-A', snow_depth_LTET3, '-B', ncalm_understory,'-C', CHM_LT1pt5,'--NoDataValue','-9999',
 '--outfile', flight_spec_understory, '--calc="((A-C)>0.3)*B"','--overwrite']
subprocess.run(SD_tall_cmd)

SD_tall_cmd = ['python',gdal_calc, '-A', flight_spec_understory, '-B', ncalm_tall,'--NoDataValue','0',
 '--outfile', flight_spec_tall, '--calc="1*logical_or(A>0, B>0)"','--overwrite']
subprocess.run(SD_tall_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/veg_classes/NCALM_SCB_20080210_understory.tif', '-B', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/NCALM_SCB_2014_tall.tif', '--NoDataValue', '0', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/veg_classes/NCALM_SCB_20080210_tall.tif', '--calc="1*logical_or(A>0, B>0)"', '--overwrite'], returncode=0)

In [385]:
toc = time.perf_counter()

### Calculate Snow Depth

In [386]:
tic=time.perf_counter()

In [348]:
# snow_depth_all_pts = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/snow_depth/ASO_SCB_20160518_SD_all.tif'
# snow_depth_open = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/snow_depth/ASO_SCB_20160518_SD_open.tif'
# snow_depth_tall = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/snow_depth/ASO_SCB_20160518_SD_tall.tif'

In [387]:
snow_depth_all_pts = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/snow_depth/NCALM_SCB_20080210_SD_all.tif'
snow_depth_open = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/snow_depth/NCALM_SCB_20080210_SD_open.tif'
snow_depth_tall = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/snow_depth/NCALM_SCB_20080210_SD_tall.tif'

In [388]:
SD_open_cmd = ['python',gdal_calc, '-A', snow_depth_LTET5, '-B', flight_spec_open,'--NoDataValue','-9999',
 '--outfile', snow_depth_open, '--calc="A*B"','--overwrite']
subprocess.run(SD_open_cmd)

SD_tall_cmd = ['python',gdal_calc, '-A', snow_depth_LTET3, '-B', flight_spec_tall,'--NoDataValue','-9999',
 '--outfile', snow_depth_tall, '--calc="A*B"','--overwrite']
subprocess.run(SD_tall_cmd)

merge_command = ["python", gdal_merge, "-o", snow_depth_all_pts, snow_depth_open, snow_depth_tall,'-a_nodata','-9999']
subprocess.call(merge_command)

0

## Filter Data
No we filter the snow depths in two critical ways.<br> From Kostadinov et al, 2019 "by excluding areas
and improving the accuracy of the lidar-derived elevation datasets. Slopes greater than 30o and lake/pond water bodies as delineated by the National Hydrography Dataset (high resolution) (https://nhd.usgs.gov/) were excluded from the analysis. We used this slope threshold because the uncertainty in elevation increases dramatically above 30 degrees slope (Takahashi et al., 2005; Tinkham et al., 2012)."

In [9]:
# Filter out all water bodies
input_shp = 'SCB/supporting_files/masks/NHDWaterbody_2018_EPSG26910.shp'

In [509]:
# input_tif = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/veg_characteristics/NCALM_SCB_2014_DNC.tif'

In [511]:
# burn_command = ['gdal_rasterize', '-b', '1', '-burn', '-9999', input_shp, input_tif]
# subprocess.call(burn_command)

In [ ]:
burn_command = ['gdal_rasterize', '-b', '1', '-burn', '-9999', input_shp, snow_depth_open]
subprocess.call(burn_command)
burn_command = ['gdal_rasterize', '-b', '1', '-burn', '-9999', input_shp, snow_depth_all_pts]
subprocess.call(burn_command)
burn_command = ['gdal_rasterize', '-b', '1', '-burn', '-9999', input_shp, snow_depth_tall]
subprocess.call(burn_command)

0

In [ ]:
# snow_depth_all_pts = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/snow_depth/ASO_SCB_20160326_SD_all.tif'
# snow_depth_filSlope = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/snow_depth/ASO_SCB_20160326_SD_filSlope.tif'
# snow_depth_refined = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/snow_depth/ASO_SCB_20160326_SD.tif'

In [ ]:
snow_depth_all_pts = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/snow_depth/NCALM_SCB_20080210_SD_all.tif'
snow_depth_filSlope = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/snow_depth/NCALM_SCB_20080210_SD_filSlope.tif'
snow_depth_refined = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/snow_depth/NCALM_SCB_20080210_SD.tif'

In [398]:
sd_filter_cmd = ['python',gdal_calc, '-A', snow_depth_all_pts,'-B',slope,'--NoDataValue','0',
 '--outfile', snow_depth_filSlope, '--calc="A*(B<=30)"','--overwrite']
subprocess.run(sd_filter_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/snow_depth/NCALM_SCB_20080210_SD_all.tif', '-B', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/DEM/SCB/NCALM_2014_slope.tif', '--NoDataValue', '0', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/snow_depth/NCALM_SCB_20080210_SD_filSlope.tif', '--calc="A*(B<=30)"', '--overwrite'], returncode=0)

In [401]:
sd_filter_cmd = ['python',gdal_calc, '-A', snow_depth_filSlope,'--NoDataValue','-9999',
 '--outfile', snow_depth_refined, '--calc="A*(A>=0.15)"','--overwrite']
subprocess.run(sd_filter_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/snow_depth/NCALM_SCB_20080210_SD_filSlope.tif', '--NoDataValue', '-9999', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/snow_depth/NCALM_SCB_20080210_SD.tif', '--calc="A*(A>=0.15)"', '--overwrite'], returncode=0)

### Create Vegetation Buffer
for statistical analysis later

In [466]:
src_tall = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/veg_classes/ASO_SCB_20160326_tall.tif'
buffer_tall = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/veg_classes/ASO_SCB_20160326_tall_buffered.tif'

In [435]:
# src_tall = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/veg_classes/NCALM_SCB_20080210_tall.tif'
# buffer_tall = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/veg_classes/NCALM_SCB_20080210_tall_buffered.tif'

In [436]:
proximity_cmd = ['python',gdal_proximity, src_tall, buffer_tall,'-values','1','-maxdist','1','-nodata','1','-fixed-buf-val','0']
subprocess.run(proximity_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_proximity.py', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/veg_classes/NCALM_SCB_20080210_tall.tif', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/veg_classes/NCALM_SCB_20080210_tall_buffered.tif', '-values', '1', '-maxdist', '1', '-nodata', '1', '-fixed-buf-val', '0'], returncode=0)

In [467]:
src_open = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/veg_classes/ASO_SCB_20160326_open.tif'
buffer_open = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/veg_classes/ASO_SCB_20160326_open_buffered.tif'

In [468]:
# src_open = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/veg_classes/NCALM_SCB_20080210_open.tif'
# buffer_open = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/veg_classes/NCALM_SCB_20080210_open_buffered.tif'

In [469]:
buffer_cmd = ['python',gdal_calc, '-A', src_open,'-B',buffer_tall,'--NoDataValue','0',
 '--outfile', buffer_open, '--calc="A*B"','--overwrite']
subprocess.run(buffer_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/veg_classes/ASO_SCB_20160326_open.tif', '-B', 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/veg_classes/ASO_SCB_20160326_tall_buffered.tif', '--NoDataValue', '0', '--outfile', 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/veg_classes/ASO_SCB_20160326_open_buffered.tif', '--calc="A*B"', '--overwrite'], returncode=0)

## Calculate SWE

Start by making sure our density rasters align with snow depth rasters <br>
from the Lidar Processing workflow we can extract our bounds: '730235.96, 738826.45,4364741.66,4372273.16'

### Align Rasters

In [440]:
input_density = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/snow_density/20160518_1m.tif'
output_density = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/snow_density/ASO_SCB_20160518_snowDensity.tif'

In [441]:
# input_density = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/snow_density/density_20080210_1m.tif'
# output_density = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/snow_density/NCALM_SCB_20080210_snowDensity.tif'

In [442]:
warp_cmd = ["gdalwarp","-overwrite", input_density, output_density, "-te", '730235.96', '4364741.66', '738826.9599999999627471', '4372273.16', '-tr','1','1','-dstnodata','0']
subprocess.call(warp_cmd)

0

### Calculate SWE

In [487]:
input_snow_depth =  'SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/snow_depth/ASO_SCB_20160326_SD.tif'
input_snow_depth_open=  'SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/snow_depth/ASO_SCB_20160326_SD_open.tif'
input_snow_depth_tall =  'SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/snow_depth/ASO_SCB_20160326_SD_tall.tif'

input_snow_density = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/snow_density/ASO_SCB_20160326_snowDensity.tif'

output_SWE =  'SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/SWE/ASO_SCB_20160326_SWE.tif'
output_SWE_open =  'SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/SWE/ASO_SCB_20160326_SWE_open.tif'
output_SWE_tall =  'SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/SWE/ASO_SCB_20160326_SWE_tall.tif'

In [532]:
# input_snow_depth =  'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/snow_depth/NCALM_SCB_20080210_SD.tif'
# input_snow_depth_open =  'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/snow_depth/NCALM_SCB_20080210_SD_open.tif'
# input_snow_depth_tall =  'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/snow_depth/NCALM_SCB_20080210_SD_tall.tif'

# input_snow_density = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/snow_density/NCALM_SCB_20080210_snowDensity.tif'

# output_SWE =  'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/SWE/NCALM_SCB_20080210_SWE.tif'
# output_SWE_open =  'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/SWE/NCALM_SCB_20080210_SWE_open.tif'
# output_SWE_tall =  'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/SWE/NCALM_SCB_20080210_SWE_tall.tif'

In [533]:
swe_cmd = ['python',gdal_calc, '-A', input_snow_depth,'-B',input_snow_density,'--NoDataValue','-9999',
 '--outfile', output_SWE, '--calc="A*B*100"','--overwrite']
subprocess.run(swe_cmd)

swe_open_cmd = ['python',gdal_calc, '-A', input_snow_depth_open,'-B',input_snow_density,'--NoDataValue','-9999',
 '--outfile', output_SWE_open, '--calc="A*B*100"','--overwrite']
subprocess.run(swe_open_cmd)

swe_tall_cmd = ['python',gdal_calc, '-A', input_snow_depth_tall,'-B',input_snow_density,'--NoDataValue','-9999',
 '--outfile', output_SWE_tall, '--calc="A*B*100"','--overwrite']
subprocess.run(swe_tall_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/snow_depth/NCALM_SCB_20080210_SD_tall.tif', '-B', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/snow_density/NCALM_SCB_20080210_snowDensity.tif', '--NoDataValue', '-9999', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/SWE/NCALM_SCB_20080210_SWE_tall.tif', '--calc="A*B*100"', '--overwrite'], returncode=0)

#### Apply Buffer Zones

In [534]:
# input_SWE =  'SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/SWE/ASO_SCB_20160326_SWE.tif'
# buffer_open = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/veg_classes/ASO_SCB_20160326_open_buffered.tif'
# SWE_open_buffered =  'SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/SWE/ASO_SCB_20160326_SWE_open_buffered.tif'

In [535]:
input_SWE =  'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/SWE/NCALM_SCB_20080210_SWE.tif'
buffer_open = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/veg_classes/NCALM_SCB_20080210_open_buffered.tif'
SWE_open_buffered =  'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/SWE/NCALM_SCB_20080210_SWE_open_buffered.tif'

In [536]:
swe_open_cmd = ['python',gdal_calc, '-A', input_SWE,'-B',buffer_open,'--NoDataValue','-9999',
 '--outfile', SWE_open_buffered, '--calc="A*B"','--overwrite']
subprocess.run(swe_open_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/SWE/NCALM_SCB_20080210_SWE.tif', '-B', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/veg_classes/NCALM_SCB_20080210_open_buffered.tif', '--NoDataValue', '-9999', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/SWE/NCALM_SCB_20080210_SWE_open_buffered.tif', '--calc="A*B"', '--overwrite'], returncode=0)

### Resample

In [553]:
# input_tif = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/SWE/ASO_SCB_20160518_SWE_open_buffered.tif'
# output_tif = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/SWE/ASO_SCB_20160518_SWE_open_buffered_30m.tif'
input_tree_tif = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/SWE/ASO_SCB_20160326_SWE_tall.tif'
output_tree_tif = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/SWE/ASO_SCB_20160326_SWE_tall_30m.tif'

In [571]:
# open_input_tif = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/SWE/NCALM_SCB_20080210_SWE_open_buffered.tif'
# open_output_tif = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/SWE/NCALM_SCB_20080210_SWE_open_buffered_30m.tif'
input_tif = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/SWE/NCALM_SCB_20080210_SWE.tif'
output_tif = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/SWE/NCALM_SCB_20080210_SWE_30m.tif'

In [572]:
warp_cmd = ["gdalwarp","-overwrite", input_tif, output_tif, "-tr", str(30),str(30),'-srcnodata','-9999','-dstnodata','-9999','-r','average']
subprocess.call(warp_cmd)

0

2008

In [38]:
# open_input_tif = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/SWE/NCALM_SCB_20080210_SWE_open_buffered.tif'
# open_output_tif = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/SWE/NCALM_SCB_20080210_SWE_open_buffered_10m.tif'
# input_tif = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/SWE/NCALM_SCB_20080210_SWE.tif'
# output_tif = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/SWE/NCALM_SCB_20080210_SWE_10m.tif'
# input_tif = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/SWE/NCALM_SCB_20080210_SWE_tall.tif'
# output_tif = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/SWE/NCALM_SCB_20080210_SWE_tall_10m.tif'

2016

In [24]:
# input_tif = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160417/SWE/ASO_SCB_20160417_SWE.tif'
# output_tif = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160417/SWE/ASO_SCB_20160417_SWE_10m.tif'
# input_tif = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160417/SWE/ASO_SCB_20160417_SWE_open_buffered.tif'
# output_tif = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160417/SWE/ASO_SCB_20160417_SWE_open_buffered_10m.tif'
# input_tif = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160417/SWE/ASO_SCB_20160417_SWE_tall.tif'
# output_tif = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160417/SWE/ASO_SCB_20160417_SWE_tall_10m.tif'

Terrain

In [36]:
input_tif = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/DEM/SCB/NCALM_2014_BE.tif'
output_tif = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/DEM/SCB/NCALM_2014_BE_10m.tif'

input_tif = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/DEM/SCB/NCALM_2014_Nness.tif'
output_tif = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/DEM/SCB/NCALM_2014_Nness_10m.tif'

In [39]:
warp_cmd = ["gdalwarp","-overwrite", input_tif, output_tif, "-tr", str(10),str(10),'-srcnodata','-9999','-dstnodata','-9999','-r','average']
subprocess.call(warp_cmd)

0

In [37]:
warp_cmd = ["gdalwarp","-overwrite", input_tif, output_tif, "-tr", str(10),str(10),'-srcnodata','-9999','-dstnodata','-9999','-r','average']
subprocess.call(warp_cmd)

0